In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import  Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train=keras.utils.image_dataset_from_directory(
    directory='./train',labels='inferred',label_mode='int',batch_size=32,image_size=(256,256)
)

test=keras.utils.image_dataset_from_directory(
    directory='./test',labels='inferred',label_mode='int',batch_size=32,image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


Normalize the above numpy array into (0-1)

In [3]:
def process(image,label):
    image=tf.cast(image/256.,tf.float32)
    return image,label
train=train.map(process)
test=test.map(process)

CNN Model

In [4]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [5]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [6]:
history=model.fit(train,epochs=10,validation_data=test)

Epoch 1/10
625/625 [==============================] - 2441s 4s/step - loss: 0.6213 - accuracy: 0.6428 - val_loss: 0.5131 - val_accuracy: 0.7496
Epoch 2/10
625/625 [==============================] - 6206s 10s/step - loss: 0.4636 - accuracy: 0.7775 - val_loss: 0.4540 - val_accuracy: 0.7806
Epoch 3/10
625/625 [==============================] - 3009s 5s/step - loss: 0.3561 - accuracy: 0.8379 - val_loss: 0.4730 - val_accuracy: 0.8064
Epoch 4/10
625/625 [==============================] - 2271s 4s/step - loss: 0.2278 - accuracy: 0.9031 - val_loss: 0.6141 - val_accuracy: 0.7926
Epoch 5/10
625/625 [==============================] - 2064s 3s/step - loss: 0.1248 - accuracy: 0.9521 - val_loss: 0.7906 - val_accuracy: 0.7928
Epoch 6/10
625/625 [==============================] - 2708s 4s/step - loss: 0.0739 - accuracy: 0.9727 - val_loss: 1.0469 - val_accuracy: 0.7860
Epoch 7/10
446/625 [====================>.........] - ETA: 11:24 - loss: 0.0611 - accuracy: 0.9792

In [ ]:
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

### Testing

In [ ]:
#test_img = cv2.imread('./dog.jpg')
test_img = cv2.imread('./cat.jpg')
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img,(256,256))
test_input = test_img.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)

### Data Augmentation

In [ ]:
dataGen=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,horizontal_flip=True,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,fill_mode='nearest')

In [ ]:
i = 0
for batch in dataGen.flow(test_input, batch_size=1,save_to_dir='aug', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break